In [ ]:
pip install sentence-transformers numpy python-dotenv chromadb

Extract Text using pypdf

In [ ]:
!pip install pypdf


In [25]:
from pypdf import PdfReader

def extract_text(path):
  reader = PdfReader(path)
  text = ""
  for page in reader.pages:
    text += page.extract_text() + "\n"
  return text


In [26]:
text = extract_text_from_pdf("/content/The Finance Bill 2025.pdf")

In [27]:
text

'   \nSPECIAL ISSUE \nNATION I. \'01.INCI I.FOR \nk P OR TI NC \nLIPRAPY  \n \n   \nKenya Gazette Supplement No. 63 (National Assembly Bills No. 19) \nREPUBLIC OF KENYA \nKENYA GAZETTE SUPPLEMENT \nNATIONAL ASSEMBLY BILLS, 2025 \nNAIROBI, 6th May, 2025 \nCONTENT \nBill for Introduction into the National Assembly— \nPAGE \nThe Finance Bill, 2025  \t 335 \nNATIONAL COUNCIL FOR \nLAW REPORTING \n0 9 MAY 2025 \nLIBRARY ARCHIVE \nPRINTED AND PUBLISHED BY THE GOVERNMENT PRINTER, NAIROBI \n335 \nTHE FINANCE BILL, 2025 \nA Bill for \nAN ACT of Parliament to amend the laws relating to \nvarious taxes and duties; and for matters incidental \nthereto \nENACTED by the Parliament of Kenya, as follows — \nPART I--PRELIMINARY \n1. This Act may be cited as the Finance Act, 2025 \nand shall come into operation as follows — \n(a) sections 12 and 56, on the 1st of January, 2026; \nand \n(b) all other sections, on 1st July, 2025. \nPART II—INCOME TAX \n2. Section 2 of the Income Tax Act is amended—\n(a) i

Step 2: Chunking

In [43]:
def chunk_text(text, size=800):
  words = text.split()
  chunks=[]
  for i in range(0, len(words), size):
    chunks.append(" ".join(words[i:i+size]))
  return chunks


In [ ]:
chunks = chunk_text(text)
chunks

In [45]:
words=text.split()
words
print(len(words))

43071


In [47]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-small-en-v1.5")

embeddings = model.encode(
    chunks,
    normalize_embeddings=True
)

print(len(embeddings), len(embeddings[0]))


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


54 384


In [48]:
finance_embeddings=model.encode(chunks).tolist()

In [ ]:
finance_embeddings

In [50]:
len(finance_embeddings)

54

In [ ]:
pip install chromadb

In [51]:
import chromadb
client=chromadb.PersistentClient(path='./chromadb')

In [53]:
collection = client.get_or_create_collection(
    name='finance_collections',
    metadata={'description': 'my_finance_collection'}
)
print('collection_created:', collection.name)

collection_created: finance_collections


In [55]:
ids=[f"doc_{i}" for i in range(len(chunks))]

In [56]:
ids

['doc_0',
 'doc_1',
 'doc_2',
 'doc_3',
 'doc_4',
 'doc_5',
 'doc_6',
 'doc_7',
 'doc_8',
 'doc_9',
 'doc_10',
 'doc_11',
 'doc_12',
 'doc_13',
 'doc_14',
 'doc_15',
 'doc_16',
 'doc_17',
 'doc_18',
 'doc_19',
 'doc_20',
 'doc_21',
 'doc_22',
 'doc_23',
 'doc_24',
 'doc_25',
 'doc_26',
 'doc_27',
 'doc_28',
 'doc_29',
 'doc_30',
 'doc_31',
 'doc_32',
 'doc_33',
 'doc_34',
 'doc_35',
 'doc_36',
 'doc_37',
 'doc_38',
 'doc_39',
 'doc_40',
 'doc_41',
 'doc_42',
 'doc_43',
 'doc_44',
 'doc_45',
 'doc_46',
 'doc_47',
 'doc_48',
 'doc_49',
 'doc_50',
 'doc_51',
 'doc_52',
 'doc_53']

In [57]:
collection.add(
    documents=chunks,
    embeddings=finance_embeddings,
    ids=ids
)

In [58]:
collection.count()

144

In [59]:
query='what does the finance bill say about taxes'

In [60]:
query_embedding = model.encode([query]).tolist()

In [61]:
query_embedding

[[-0.051833897829055786,
  -0.04461318626999855,
  -0.03269420191645622,
  0.00829776655882597,
  0.03347180783748627,
  -0.020946398377418518,
  0.060635559260845184,
  -0.0110707338899374,
  0.02748493105173111,
  0.0038460486102849245,
  0.04119643196463585,
  0.06742852181196213,
  -0.04187445715069771,
  0.0124196233227849,
  -0.023866765201091766,
  -0.018870240077376366,
  0.041657742112874985,
  -0.05052805319428444,
  -0.10898711532354355,
  0.06502988934516907,
  0.10386475175619125,
  -0.01792837493121624,
  -0.012100756168365479,
  -0.00835898146033287,
  0.02158215269446373,
  0.01483629085123539,
  -0.03328530490398407,
  -0.025262119248509407,
  -0.03794164955615997,
  -0.11857001483440399,
  0.00896485149860382,
  -0.04740777984261513,
  -0.019124310463666916,
  -0.02610534243285656,
  0.09477393329143524,
  0.013380511663854122,
  0.0006159348413348198,
  -0.011326986365020275,
  0.004988922271877527,
  0.045586295425891876,
  -0.008696085773408413,
  0.055599164217710

In [64]:
query_embedding = model.encode([query]).tolist()
results=collection.query(
    query_embeddings=query_embedding,
    n_results=3
)
results

{'ids': [['doc_18', 'doc_42', 'doc_54']],
 'embeddings': None,
 'documents': [['in Kenya, or held on behalf of the taxpayer or non-resident person .who is subject to tax in. Kenya"; (f) in subsection (6), by inserting the words "or the non-resident person who is subject to tax in Kenya" immediately after the word "taxpayer"; (i) 356 The Finance Bill, 2025 (g) in subsection (8), by inserting the words "or the non-resident person who is subject to tax in Kenya" immediately after the word "taxpayer"; (h) in subsection (9), by inserting the words "or the non-resident person who is subject to tax in Kenya" immediately after the word "taxpayer"; (i) in subsection (10), by inserting the words "or the non-resident person who is subject to tax in Kenya" immediately after the word "taxpayer" wherever it occurs; (j) in subsection (11), by inserting the words "or the non-resident person who is subject to tax in Kenya" immediately after the word "taxpayer"; (k) in subsection (12), by inserting the 